In [2]:
import os
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import Callback
import zipfile
import os
import gdown

In [ ]:
#Google Drive file URL
file_url = "https://drive.google.com/uc?id=1vHCvd9ZFkY9lfNwUnnMUbDQZjjpby8lv"

#Specify the output file path (the path where you want to save the file)
output_file = "disease.zip"  # Replace with actual file name

#Download the file
gdown.download(file_url, output_file, quiet=False)

zip_file = "disease.zip"  #Name of the downloaded file

#Make sure output directory exists
output_dir = "/content/extracted_file"
os.makedirs(output_dir, exist_ok=True)

#Extract the contents
with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall(output_dir)

Downloading...
From (original): https://drive.google.com/uc?id=1QwItNnETzlWHgZ89EbHFYg04ijkDszF6
From (redirected): https://drive.google.com/uc?id=1QwItNnETzlWHgZ89EbHFYg04ijkDszF6&confirm=t&uuid=fbd116af-370c-499a-be01-55dffa6bda58
To: /content/disease.zip
100%|██████████| 3.85G/3.85G [00:55<00:00, 68.9MB/s]


In [ ]:
#Path to main train folder that has many class folders
data_dir = "/content/extracted_file/Retinal Fundus Images - Copy/train"

#Specify the exact classes you want to include
target_classes = ['Glaucoma', 'Normal_Fundus']

#Image settings
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

# Create generators with only selected classes
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.1)

train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    classes=target_classes,  #This filters folders
    class_mode='sparse',
    subset='training',
    shuffle=True
)

val_generator = datagen.flow_from_directory(
    data_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    classes=target_classes,  #This filters folders
    class_mode='sparse',
    subset='validation',
    shuffle=False
)


#Build ResNet50-based model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(128, activation='relu')(x)
output = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=output)

#Fine-tune last 30 layers of ResNet50
for layer in base_model.layers[:-45]:
    layer.trainable = False
for layer in base_model.layers[-45:]:
    layer.trainable = True

#Compile with smaller learning rate for fine-tuning
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss='binary_crossentropy',
              metrics=['accuracy'])


#Train the model
model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,
)

Found 19436 images belonging to 2 classes.
Found 2159 images belonging to 2 classes.
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
608/608 ━━━━━━━━━━━━━━━━━━━━ 130s 169ms/step - accuracy: 0.7664 - loss: 0.4750 - val_accuracy: 0.7925 - val_loss: 0.4053
Epoch 2/10
608/608 ━━━━━━━━━━━━━━━━━━━━ 88s 145ms/step - accuracy: 0.8625 - loss: 0.3093 - val_accuracy: 0.8300 - val_loss: 0.3350
Epoch 3/10
608/608 ━━━━━━━━━━━━━━━━━━━━ 89s 147ms/step - accuracy: 0.8891 - loss: 0.2572 - val_accuracy: 0.8703 - val_loss: 0.2778
Epoch 4/10
608/608 ━━━━━━━━━━━━━━━━━━━━ 142s 147ms/step - accuracy: 0.9136 - loss: 0.2099 - val_accuracy: 0.8536 - val_loss: 0.3213
Epoch 5/10
608/608 ━━━━━━━━━━━━━━━━━━━━ 92s 151ms/step - accuracy: 0.9298 - loss: 0.1699 - val_accuracy: 0.8277 - val_loss: 0.3769
Epoch 6/10
608/608 ━━━━━━━━━━━━━━━━━━━━ 91s 150ms/step - accuracy: 0.9397 - loss: 0.1458 - val_accuracy: 0.8648 - val_loss: 0.3108
Epoch 7/10
608/608 ━━━━━━━━━━━━━━━━━━━━ 91s 149ms/step - accuracy: 0.9513 - loss: 0.1229 - val_accuracy: 0.9301 - val_loss: 0.1671
Epoch 8/10
608/608 ━━━━━━━━━━━━━━━━━━━━ 91s 150ms/step - accuracy: 0.9572 - loss:

In [5]:
model.save("Glaucoma.keras")